# Recipe Dataset (over 2M) Food

This dataset is a comprehensive collection of recipes from all around the world, ranging from simple dishes like bread to elaborate meals like Swedish midsummer smorgasbords. It is designed to facilitate projects that involve food analysis, recipe generation, or multimedia applications related to culinary arts.

## Used libraries

In [ ]:
# General-purpose libraries
import pandas as pd
import numpy as np
import seaborn as sns

# Text-to-Video and NLP Libraries
from transformers import pipeline
from diffusers import StableDiffusionPipeline

# Video and Image Handling
from moviepy import ImageSequenceClip, AudioFileClip
import cv2  # OpenCV for image manipulation

# Text-to-Speech
from gtts import gTTS

# Miscellaneous
import os

## Constants

In [ ]:
df_columns = df.columns

## Helper methods

In [ ]:
# Function to print dataset.
def print_dataset(text, df):
    print("\n" + text + ":")
    display(df.head())

# Check for noisy data (e.g., special characters or unnecessary brackets)
def find_noisy_data(column):
    noisy_rows = df[column][df[column].str.contains(r"[\\[\\]\\\\]|\\\"")]
    return noisy_rows

## Read Dataset

In [ ]:
# Read dataset
df = pd.read_csv("../files/recipes_data.csv")
# df = pd.read_csv("../files/processed_data.csv")

## Model preparation

### Database structure

In [6]:
print_dataset("Dataset", df)


Dataset:


,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com


### Data types

In [7]:
# To gain knowledge about data types, run this command:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   title        object
 1   ingredients  object
 2   directions   object
 3   link         object
 4   source       object
 5   NER          object
 6   site         object
dtypes: object(7)
memory usage: 119.2+ MB


### NULL values

In [ ]:
# Command for checking for null values:
df.isnull().sum()

title          1
ingredients    0
directions     0
link           0
source         0
NER            0
site           0
dtype: int64

- Handling null values

In [ ]:
# Fshirja e rreshtave me vlera null në kolonat me vlera null
df = df.dropna(subset=df_columns)
#new_df.to_csv("../files/Preprocessed_Kosovo_News_Articles_Dataset.csv", index=False)

# Shfaqja e dataseti-it të modifikuar
print(df.isnull().sum())

###Duplicate values

In [ ]:
- Duplicate values in dataset

# Command to search duplicates
print("Duplicates: " + str(df.duplicated().sum()))

- Duplicate values in title column

In [ ]:
duplicates = df[df['title'].duplicated(keep=False)]
print(duplicates)

- Rows filter based on the title

In [ ]:
# Filter the DataFrame for rows where the title is "Cherry Nut Bars"
cherry_nut_bars = df[df['title'] == "Cherry Nut Bars"]

# Display the filtered rows
# print_dataset("cherry_nut_bars", cherry_nut_bars)
print(cherry_nut_bars.to_string())


- Find duplicates in NER column

In [ ]:
duplicates = df[df['NER'].duplicated(keep=False)]
print(duplicates)


- Removing nearly duplicate values

In [ ]:
# Number of rows before removing duplicates
rows_before = len(df)

# Identify duplicates based on 'title', 'NER', and 'ingredients'
duplicates = df[df.duplicated(subset=['title', 'NER', 'ingredients'], keep=False)]

# Log duplicate rows for verification
print("Duplicate Rows:")
print(duplicates)

# Remove duplicates, keeping only the first occurrence
df_cleaned = df.drop_duplicates(subset=['title', 'NER'], keep='first')

# Number of rows after removing duplicates
rows_after = len(df_cleaned)

# Calculate the number of deleted rows
deleted_rows = rows_before - rows_after

df = df_cleaned

# Logs
print(f"\nRows before removing duplicates: {rows_before}")
print(f"Rows after removing duplicates: {rows_after}")
print(f"Number of rows deleted: {deleted_rows}")


- Rows after removing nearly duplicates

In [ ]:
# Filter the DataFrame for rows where the title is "Cherry Nut Bars"
cherry_nut_bars = df[df['title'] == "Cherry Nut Bars"]

# Display the filtered rows
# print_dataset("cherry_nut_bars", cherry_nut_bars)
print(cherry_nut_bars.to_string())

### NaN values

In [ ]:
# Iterate through each column in the DataFrame
for column in df_columns:
    nan_count = df[column].isna().sum()  # Count missing (NaN) values in the column
    print(f"The number of missing values detected in the column '{column}' is: {nan_count}")

- Handling NaN values

In [ ]:
df = df.dropna(axis=0, how='any')

### Drop columns

In [ ]:
# Delete columns 'City' and 'Salary'
columns_to_delete = ['link', 'source', 'site']
df.drop(columns=columns_to_delete, inplace=True)

In [ ]:
### Special characters

In [ ]:
# Identify titles with special characters
print("Titles with special characters:")
print(df[df['title'].str.contains(r'[^\w\s]', regex=True)])

- Handling special characters

In [ ]:
# Remove special characters from titles
df['title'] = df['title'].str.replace(r'[^\w\s]', '', regex=True)

### Noisy data

In [ ]:
# Display noisy data in the 'ingredients' column
noisy_data = find_noisy_data('ingredients')
print("Noisy Data in 'ingredients':")
print(noisy_data)

In [ ]:
# Display noisy data in the 'directions' column
noisy_data = find_noisy_data('directions')
print("Noisy Data in 'directions':")
print(noisy_data)

#### Handling Noisy Data

In [ ]:
# Clean noisy data in the 'ingredients' column
df['ingredients'] = df['ingredients'] \
    .str.replace(r'\\"', '"', regex=True) \
    .str.replace(r'[\[\]]', '', regex=True) \
    .str.replace(r'\\', '', regex=True) \
    .str.strip()

# Verify the cleaned column
print("Cleaned Ingredients Column:")
print(df['ingredients'].head())

In [ ]:
# Clean noisy data in the 'ingredients' column
df['directions'] = df['directions'] \
    .str.replace(r'\\"', '"', regex=True) \
    .str.replace(r'[\[\]]', '', regex=True) \
    .str.replace(r'\\', '', regex=True) \
    .str.strip()

# Verify the cleaned column
print("Cleaned Ingredients Column:")
print(df['directions'].head())

### Outliers

In [ ]:
# Count the frequency of each unique string
frequency_counts = df['title'].value_counts()

# Define a threshold for rare occurrences (e.g., frequency = 1)
outliers = frequency_counts[frequency_counts == 1]

# Display outliers
print("Outliers based on frequency:")
print(outliers)

- Find groups data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Krijimi i një TF-IDF vektorizuesi
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['title'])

# Klasterizimi me K-Means
num_clusters = 3  # Numri i grupeve
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Shtimi i grupeve për secilin titull
clusters = kmeans.labels_

# Shfaqja e rezultateve
for title, cluster in zip(df['title'], clusters):
    print(f"'{title}' është në grupin {cluster}")

- Clustering recipe titles by category

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd

# Get 50% of the data
df = df.sample(frac=0.5, random_state=42)

# Reset the index to align with PCA results
df = df.reset_index(drop=True)

# Handle missing values
df['title'] = df['title'].fillna('')
df['ingredients'] = df['ingredients'].fillna('')

# TF-IDF Vectorization on Titles
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['title'])

print(f"TF-IDF Matrix Shape: {X.shape}")

# K-Means Clustering
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Assign Cluster Labels
clusters = kmeans.labels_
df['Cluster'] = clusters

# Manually Define Categories for Clusters
cluster_names = {
    0: 'Dessert Recipes',
    1: 'Dinner Recipes',
    2: 'Breakfast Recipes'
}

# Assign Category Names
df['Category'] = df['Cluster'].map(cluster_names)

# Print Grouped Titles by Cluster
print("\n--- Titles Grouped by Cluster ---")
for cluster, category in cluster_names.items():
    print(f"\n{category}:")
    titles_in_group = df[df['Cluster'] == cluster]['title'].tolist()
    for title in titles_in_group:
        print(f"- {title}")

# Dimensionality Reduction with PCA for Visualization
pca = PCA(n_components=2, random_state=42)
X_reduced = pca.fit_transform(X.toarray())
print(f"PCA Reduced Shape: {X_reduced.shape}")

# Visualization with Category Names
plt.figure(figsize=(10, 7))
for cluster, category in cluster_names.items():
    points = X_reduced[df['Cluster'] == cluster]
    plt.scatter(points[:, 0], points[:, 1], label=category)

# Removed Add Titles as Labels (plt.text)
# for i, (x, y) in enumerate(X_reduced):
#     plt.text(x, y, df.iloc[i]['title'], fontsize=8)

plt.title("Clustering of Recipe Titles by Category")
plt.xlabel("Title")
plt.ylabel("Category")
plt.legend()
plt.show()

### NLP Video Generation

In [ ]:
from diffusers import StableDiffusionPipeline
import os

# Load Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cpu")  # Use CPU instead of GPU

# Create a directory for generated images
os.makedirs("generated_frames", exist_ok=True)

print("Stable Diffusion model loaded successfully in CPU mode!")

#### Generate Images for Recipe Steps

In [ ]:
# Reset the index
sample_df = sample_df.reset_index(drop=True)

# Select the first recipe
recipe_title = sample_df['title'][0]
recipe_steps = sample_df['directions'][0]  # This is already a list

frames = []  # To store generated image paths
for i, step in enumerate(recipe_steps):
    prompt = f"Artistic representation of: {step}"
    image = pipe(prompt).images[0]
    frame_path = f"generated_frames/{recipe_title.replace(' ', '_')}_step_{i+1}.png"
    image.save(frame_path)
    frames.append(frame_path)

print(f"Generated {len(frames)} images for '{recipe_title}' recipe.")

#### Combine Frames into a Video

In [ ]:
import os
import imageio_ffmpeg

ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
print("FFmpeg binary path:", ffmpeg_path)

# Explicitly set ffmpeg path
os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

# Verify the ffmpeg path
print("Using FFmpeg binary at:", imageio_ffmpeg.get_ffmpeg_exe())

In [ ]:
# Directory containing frames
frame_directory = "generated_frames"

# Get all image files in the directory
all_frames = sorted([f for f in os.listdir(frame_directory) if f.endswith(".png")])

# Group frames by their prefix
frame_groups = {}
for frame in all_frames:
    prefix = "_".join(frame.split("_")[:-2])  # Extract prefix (everything except step_x)
    if prefix not in frame_groups:
        frame_groups[prefix] = []
    frame_groups[prefix].append(os.path.join(frame_directory, frame))

# Create a video for each group
fps = 1  # Frames per second
output_directory = "../videos/"
os.makedirs(output_directory, exist_ok=True)

for prefix, frames in frame_groups.items():
    video_path = os.path.join(output_directory, f"{prefix}.mp4")
    clip = ImageSequenceClip(frames, fps=fps)
    clip.write_videofile(video_path, codec="libx264")
    print(f"Video saved: {video_path}")

## NER Analysis

### Segmentation

Segmentation in NLP involves breaking down a larger piece of text into smaller, meaningful units such as sentences or paragraphs.

In [ ]:
# Check the type of the `directions` column
print(type(df['directions'][0]))

# If `directions` is already a list, skip the parsing step
# Flatten and segment the directions into smaller pieces
segmented_directions = []
for index, row in df.iterrows():
    for step in row['directions']:
        # Split steps into smaller segments using `.split('. ')`
        segments = step.split('. ')
        for segment in segments:
            # Avoid adding empty strings
            if segment.strip():
                segmented_directions.append({
                    "title": row["title"],
                    "direction_segment": segment.strip()  # Clean whitespace
                })

# Convert the segmented directions into a new DataFrame
segmented_df = pd.DataFrame(segmented_directions)

# Export the segmented DataFrame to a CSV file
segmented_df.to_csv("../files/1_segmentation.csv", index=False)

# Print the first few rows for verification
print(segmented_df.head())

### Tokenization

Tokenization is used in MLP to split paragraphs and sentences into smaller units that can be more easily assigned meaning.

In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')

# Check if `directions` is a list or a string
if isinstance(df['directions'][0], str):
    import ast
    df['directions'] = df['directions'].apply(ast.literal_eval)

# Segment and tokenize the directions
segmented_tokenized_directions = []
for index, row in df.iterrows():
    for step in row['directions']:
        # Segment the step into smaller parts
        segments = step.split('. ')
        for segment in segments:
            # Clean whitespace and ensure the segment is not empty
            clean_segment = segment.strip()
            if clean_segment:
                try:
                    # Tokenize the segment into words
                    tokens = word_tokenize(clean_segment)
                    segmented_tokenized_directions.append({
                        "title": row["title"],
                        "direction_segment": clean_segment,
                        "tokens": tokens
                    })
                except Exception as e:
                    print(f"Error tokenizing: {clean_segment}. Error: {e}")

# Convert the results into a DataFrame
segmented_tokenized_df = pd.DataFrame(segmented_tokenized_directions)

# Convert the tokens column to a string for proper CSV export
segmented_tokenized_df['tokens'] = segmented_tokenized_df['tokens'].apply(lambda x: ' '.join(x))

# Export the segmented and tokenized DataFrame to a CSV file
segmented_tokenized_df.to_csv("../files/2_tokenization.csv", index=False)

# Print the first few rows for verification
print(segmented_tokenized_df.head())